### Task 0: Preparation

You need to choose **three** datasets from the UCR/UEA time series repository. Please be careful since some UCR datasets can take a long time to be processed - You do not need to choose heavy datasets since it would slow your training/testing process. Use sktime's `load_UCR_UEA_dataset` function to perform. Please note that **you should use each dataset's original train/test splits** to train and report the test scores.

In [1]:
from sktime.datasets import load_UCR_UEA_dataset
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import regularizers
from sklearn.preprocessing import LabelEncoder 
from sktime.classification.deep_learning.cnn import CNNClassifier
from keras.layers import Input, Dense, SimpleRNN, Flatten, GlobalAveragePooling1D
from tensorflow import keras
from keras import layers
from tensorflow.keras.models import Model
from sklearn.model_selection import RandomizedSearchCV
from scikeras.wrappers import KerasClassifier
from sktime.classification.compose import ClassifierPipeline
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sktime.transformations.panel.rocket import Rocket
from sktime.transformations.panel.catch22 import Catch22
from sklearn.tree import DecisionTreeClassifier
from sktime.classification.interval_based import TimeSeriesForestClassifier


In [2]:
X_train, y_train= load_UCR_UEA_dataset("GunPoint", split="train", return_type="numpy2D")
X_test, y_test= load_UCR_UEA_dataset("GunPoint", split="test", return_type="numpy2D")

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train=y_train.astype(int)
y_test=y_test.astype(int)

X1_train, y1_train = load_UCR_UEA_dataset(name='ArrowHead', split="train",return_type='numpy2D')
X1_test, y1_test = load_UCR_UEA_dataset(name= 'ArrowHead', split="test",return_type='numpy2D')

X1_train = X1_train.astype(np.float32)
X1_test = X1_test.astype(np.float32)
y1_train=y1_train.astype(float)
y1_test=y1_test.astype(float)

X2_train, y2_train = load_UCR_UEA_dataset(name='ECGFiveDays', split="train",return_type='numpy2D')
X2_test, y2_test = load_UCR_UEA_dataset(name= 'ECGFiveDays', split="test",return_type='numpy2D')

le = LabelEncoder()

X2_train = X2_train.astype(np.float32)
X2_test = X2_test.astype(np.float32)
y2_train= le.fit_transform(y2_train)
y2_test=le.fit_transform(y2_test)


In [3]:

for i in range(len(y_train)):
  if y_train[i]==1:
    y_train[i]=0
  else:
    y_train[i]=1
for i in range(len(y_test)):
  if y_test[i]==1:
    y_test[i]=0
  else:
    y_test[i]=1

np.unique(y_train),np.unique(y_test)

np.unique(y_train)

array([0, 1])

In [4]:

for i in range(len(y1_train)):
  if y1_train[i]==-1:
    y1_train[i]=0
  else:
    y1_train[i]=1
for i in range(len(y1_test)):
  if y1_test[i]==-1:
    y1_test[i]=0
  else:
    y1_test[i]=1

np.unique(y1_train),np.unique(y1_test)

(array([1.]), array([1.]))

In [5]:
datasets = {'GunPoint': (X_train, y_train, X_test, y_test), 'AroowHead': (X1_train, y1_train, X1_test, y1_test), 'ECGFiveDays': (X2_train, y2_train, X2_test, y2_test)}


### Task 1: Time series classification using deep learning 1



Time series classification problems can be solved using networks like CNN, RNN, or FCN. You can even try to merge different networks. In this task, you must test your three chosen datasets on four other models.

Try to implement four different models: 1) Fully connected network, 2) One directional RNN, 3) 1D-CNN only, and 4) 1D-CNN + LSTM. Note that you can always link the network to a fully connected layer to match the output size. You can freely construct any structure you want. Report the average test scores of four models on three datasets you chose. It would be four scores in total. Mark the best model in terms of the average test score. Briefly explain the structure you constructed. There is no definitive answer, and it is up to your own model. Here it would be best if you keep the following rules:

- When initially loading the dataset, use sktime's `load_UCR_UEA_dataset` function. This is for our grading purpose.
- You should use at least **two** Tensorflow callbacks when you fit your model. These can be built-in ones or your personalized callback.
- You should use Tensorflow's data API (`tf.data`) to manage your dataset and use `shuffle`, `batch,` and `prefetch` functions. This means that you need to convert the data format using the `from_tensor_slices` function. This also means that you need to create your own validation set. You are not limited to using any methods to do this, but you may also need to shuffle the dataset before (for that, you can use `np.random.permutation`).
- You need to clearly report the **test accuracy** of the four models. Training and validation accuracy scores are not enough.
- You may need to deal with datasets of different sizes. In this case, it might be helpful to make a function to create a model that can receive different input sizes as a parameter.


In [6]:


def fcn(ip_shape, X, y, x_test, y_test, loss_fn, act_fn):
    model = keras.models.Sequential([
        layers.Flatten(input_shape=ip_shape, name='flatten'),
        layers.Dense(128, activation='tanh', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dense(64, activation='tanh', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dense(32, activation='tanh', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dense(1, activation=act_fn)
    ])

    model.compile(loss=loss_fn, optimizer='adam', metrics=['acc'])
    model.summary()

    perm = np.random.permutation(len(X))
    X_sh, y_sh = X[perm], y[perm]

    train_size = int(0.8 * len(X_sh))
    val_size = len(X_sh) - train_size

    train_dataset = tf.data.Dataset.from_tensor_slices((X_sh[:train_size], y_sh[:train_size]))
    train_dataset = train_dataset.batch(64).prefetch(tf.data.AUTOTUNE)

    val_dataset = tf.data.Dataset.from_tensor_slices((X_sh[train_size:], y_sh[train_size:]))
    val_dataset = val_dataset.batch(64).prefetch(tf.data.AUTOTUNE)

    callback1 = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=3)
    callback2 = tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5', save_best_only=True)

    history = model.fit(train_dataset, shuffle=True, validation_data=val_dataset, epochs=30, callbacks=[callback1, callback2])

    test_loss, test_accuracy = model.evaluate(x_test, y_test)
    print("Test Loss:", test_loss)
    print("Test Accuracy:", test_accuracy)



In [7]:
fcn(X_train.shape[1:],X_train,y_train,X_test,y_test,'binary_crossentropy','tanh')

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 150)               0         
                                                                 
 dense (Dense)               (None, 128)               19328     
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 29697 (116.00 KB)
Trainable params: 29697 (116.00 KB)
Non-trainable params: 0 (0.00 Byte)
_______

2023-05-31 17:02:15.215447: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-31 17:02:15.215474: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/30


2023-05-31 17:02:15.452210: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [40]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:15.732286: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 1.2916 - acc: 0.5250

2023-05-31 17:02:16.145371: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [10]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:16.215302: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 940ms/step - loss: 1.2916 - acc: 0.5250 - val_loss: 8.0048 - val_acc: 0.5000
Epoch 2/30
1/1 [==============================] - 0s 75ms/step - loss: 5.7518 - acc: 0.5000 - val_loss: 1.2691 - val_acc: 0.6000
Epoch 3/30
1/1 [==============================] - 0s 45ms/step - loss: 1.0133 - acc: 0.6500 - val_loss: 1.2885 - val_acc: 0.5000
Epoch 4/30
1/1 [==============================] - ETA: 0s - loss: 1.0808 - acc: 0.5250

/Users/lekhmac/miniforge3/envs/tensenv/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 43ms/step - loss: 1.0808 - acc: 0.5250 - val_loss: 1.4921 - val_acc: 0.5000
Epoch 5/30
5/5 [==============================] - 0s 29ms/step - loss: 1.4258 - acc: 0.4933
Test Loss: 1.425815463066101
Test Accuracy: 0.4933333396911621


In [8]:
fcn(X1_train.shape[1:],X1_train,y1_train,X1_test,y1_test,'binary_crossentropy','tanh')

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 251)               0         
                                                                 
 dense_4 (Dense)             (None, 128)               32256     
                                                                 
 dense_5 (Dense)             (None, 64)                8256      
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dense_7 (Dense)             (None, 1)                 33        
                                                                 
Total params: 42625 (166.50 KB)
Trainable params: 42625 (166.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30


2023-05-31 17:02:16.953752: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [28]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:17.193167: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 15.7205 - acc: 0.0000e+00

2023-05-31 17:02:17.522278: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [8]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:17.571104: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 787ms/step - loss: 15.7205 - acc: 0.0000e+00 - val_loss: 15.7147 - val_acc: 0.0000e+00
Epoch 2/30
1/1 [==============================] - 0s 72ms/step - loss: 15.7147 - acc: 0.0000e+00 - val_loss: 15.7089 - val_acc: 0.0000e+00
Epoch 3/30
1/1 [==============================] - 0s 72ms/step - loss: 15.7089 - acc: 0.0000e+00 - val_loss: 15.7032 - val_acc: 0.0000e+00
Epoch 4/30
5/6 [========================>.....] - ETA: 0s - loss: 15.6116 - acc: 0.0000e+00

2023-05-31 17:02:18.058382: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


6/6 [==============================] - 0s 28ms/step - loss: 15.6190 - acc: 0.0000e+00
Test Loss: 15.618959426879883
Test Accuracy: 0.0


In [9]:
fcn(X2_train.shape[1:],X2_train,y2_train,X2_test,y2_test,'binary_crossentropy','softmax')

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 136)               0         
                                                                 
 dense_8 (Dense)             (None, 128)               17536     
                                                                 
 dense_9 (Dense)             (None, 64)                8256      
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 27905 (109.00 KB)
Trainable params: 27905 (109.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30


2023-05-31 17:02:18.423515: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [18]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:18.652083: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 0.9524 - acc: 0.3333

2023-05-31 17:02:18.978555: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [5]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:19.024945: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 775ms/step - loss: 0.9524 - acc: 0.3333 - val_loss: 1.2203 - val_acc: 0.6000
Epoch 2/30
1/1 [==============================] - 0s 47ms/step - loss: 0.8510 - acc: 0.3333 - val_loss: 1.2801 - val_acc: 0.6000
Epoch 3/30
1/1 [==============================] - 0s 47ms/step - loss: 0.7904 - acc: 0.3333 - val_loss: 1.3218 - val_acc: 0.6000
Epoch 4/30
27/27 [==============================] - 1s 17ms/step - loss: 0.9988 - acc: 0.5029
Test Loss: 0.998835563659668
Test Accuracy: 0.5029035806655884


In [10]:
def RNN(ip_shape, X, y, x_test, y_test, loss_fn, act_fn):
    model = keras.models.Sequential([
        layers.Reshape((ip_shape[0], 1), input_shape=ip_shape),
        layers.LSTM(64, input_shape=ip_shape),
        layers.Dense(1, activation=act_fn)
    ])

    model.compile(loss=loss_fn, optimizer='adam', metrics=['acc'])
    model.summary()

    perm = np.random.permutation(len(X))
    X_sh, y_sh = X[perm], y[perm]

    train_size = int(0.8 * len(X_sh))
    val_size = len(X_sh) - train_size

    train_dataset = tf.data.Dataset.from_tensor_slices((X_sh[:train_size], y_sh[:train_size]))
    train_dataset = train_dataset.batch(64).prefetch(tf.data.AUTOTUNE)

    val_dataset = tf.data.Dataset.from_tensor_slices((X_sh[train_size:], y_sh[train_size:]))
    val_dataset = val_dataset.batch(64).prefetch(tf.data.AUTOTUNE)

    callback1 = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=3)
    callback2 = tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5', save_best_only=True)

    history = model.fit(train_dataset, shuffle=True, validation_data=val_dataset, epochs=30, callbacks=[callback1, callback2])

    test_loss, test_accuracy = model.evaluate(x_test, y_test)
    print("Test Loss:", test_loss)
    print("Test Accuracy:", test_accuracy)

In [11]:
RNN(X_train.shape[1:], X_train, y_train, X_test, y_test, 'binary_crossentropy', 'sigmoid')

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 150, 1)            0         
                                                                 
 lstm (LSTM)                 (None, 64)                16896     
                                                                 
 dense_12 (Dense)            (None, 1)                 65        
                                                                 
Total params: 16961 (66.25 KB)
Trainable params: 16961 (66.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30


2023-05-31 17:02:20.237445: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [40]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:20.906546: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:02:21.116027: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:02:21.313132: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 0.6937 - acc: 0.4750

2023-05-31 17:02:21.479885: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [10]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:21.629205: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 2s 2s/step - loss: 0.6937 - acc: 0.4750 - val_loss: 0.6943 - val_acc: 0.5000
Epoch 2/30
1/1 [==============================] - ETA: 0s - loss: 0.6925 - acc: 0.5250

2023-05-31 17:02:21.719889: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 66ms/step - loss: 0.6925 - acc: 0.5250 - val_loss: 0.6963 - val_acc: 0.5000
Epoch 3/30
1/1 [==============================] - 0s 68ms/step - loss: 0.6922 - acc: 0.5250 - val_loss: 0.6970 - val_acc: 0.5000
Epoch 4/30
5/5 [==============================] - 0s 25ms/step - loss: 0.6931 - acc: 0.4933
Test Loss: 0.6931150555610657
Test Accuracy: 0.4933333396911621


In [12]:
RNN(X1_train.shape[1:], X1_train, y1_train, X1_test, y1_test, 'binary_crossentropy', 'sigmoid')

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_1 (Reshape)         (None, 251, 1)            0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                16896     
                                                                 
 dense_13 (Dense)            (None, 1)                 65        
                                                                 
Total params: 16961 (66.25 KB)
Trainable params: 16961 (66.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30


2023-05-31 17:02:22.490418: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [28]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:23.614951: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:02:23.828858: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:02:23.994556: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 0.6166 - acc: 1.0000

2023-05-31 17:02:24.131386: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [8]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:24.279927: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 2s 2s/step - loss: 0.6166 - acc: 1.0000 - val_loss: 0.5472 - val_acc: 1.0000
Epoch 2/30
1/1 [==============================] - ETA: 0s - loss: 0.5569 - acc: 1.0000

2023-05-31 17:02:24.369835: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 91ms/step - loss: 0.5569 - acc: 1.0000 - val_loss: 0.4900 - val_acc: 1.0000
Epoch 3/30
1/1 [==============================] - 0s 94ms/step - loss: 0.5017 - acc: 1.0000 - val_loss: 0.4374 - val_acc: 1.0000
Epoch 4/30
1/1 [==============================] - 0s 86ms/step - loss: 0.4506 - acc: 1.0000 - val_loss: 0.3889 - val_acc: 1.0000


2023-05-31 17:02:24.980046: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:02:25.068193: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


6/6 [==============================] - 0s 27ms/step - loss: 0.4076 - acc: 1.0000
Test Loss: 0.4075777232646942
Test Accuracy: 1.0


In [13]:
RNN(X2_train.shape[1:], X2_train, y2_train, X2_test, y2_test, 'binary_crossentropy', 'sigmoid')

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_2 (Reshape)         (None, 136, 1)            0         
                                                                 
 lstm_2 (LSTM)               (None, 64)                16896     
                                                                 
 dense_14 (Dense)            (None, 1)                 65        
                                                                 
Total params: 16961 (66.25 KB)
Trainable params: 16961 (66.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30


2023-05-31 17:02:25.487705: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [18]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:26.142801: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:02:26.361414: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:02:26.541661: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 0.7120 - acc: 0.2222

2023-05-31 17:02:26.686748: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [5]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:26.837645: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 2s 2s/step - loss: 0.7120 - acc: 0.2222 - val_loss: 0.6903 - val_acc: 0.8000
Epoch 2/30
1/1 [==============================] - ETA: 0s - loss: 0.7026 - acc: 0.2778

2023-05-31 17:02:26.929275: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 55ms/step - loss: 0.7026 - acc: 0.2778 - val_loss: 0.6960 - val_acc: 0.2000
Epoch 3/30
1/1 [==============================] - 0s 52ms/step - loss: 0.6936 - acc: 0.8333 - val_loss: 0.7018 - val_acc: 0.2000
Epoch 4/30
27/27 [==============================] - 1s 18ms/step - loss: 0.6882 - acc: 0.4983
Test Loss: 0.6881718039512634
Test Accuracy: 0.4982578158378601


In [14]:
def CNN(ip_shape, X, y, x_test, y_test, loss_fn, act_fn):
    model = keras.models.Sequential([
        layers.Reshape((ip_shape[0], 1), input_shape=ip_shape),
        layers.Conv1D(32, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation=act_fn)
    ])

    model.compile(loss=loss_fn, optimizer='adam', metrics=['acc'])
    model.summary()

    perm = np.random.permutation(len(X))
    X_sh, y_sh = X[perm], y[perm]

    train_size = int(0.8 * len(X_sh))
    val_size = len(X_sh) - train_size

    train_dataset = tf.data.Dataset.from_tensor_slices((X_sh[:train_size], y_sh[:train_size]))
    train_dataset = train_dataset.batch(64).prefetch(tf.data.AUTOTUNE)

    val_dataset = tf.data.Dataset.from_tensor_slices((X_sh[train_size:], y_sh[train_size:]))
    val_dataset = val_dataset.batch(64).prefetch(tf.data.AUTOTUNE)

    callback1 = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=3)
    callback2 = tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5', save_best_only=True)

    history = model.fit(train_dataset, shuffle=True, validation_data=val_dataset, epochs=30, callbacks=[callback1, callback2])

    test_loss, test_accuracy = model.evaluate(x_test, y_test)
    print("Test Loss:", test_loss)
    print("Test Accuracy:", test_accuracy)


In [15]:
CNN(X_train.shape[1:], X_train, y_train, X_test, y_test, 'binary_crossentropy', 'sigmoid')

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_3 (Reshape)         (None, 150, 1)            0         
                                                                 
 conv1d (Conv1D)             (None, 148, 32)           128       
                                                                 
 max_pooling1d (MaxPooling1  (None, 74, 32)            0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 2368)              0         
                                                                 
 dense_15 (Dense)            (None, 64)                151616    
                                                                 
 dense_16 (Dense)            (None, 1)                 65        
                                                      

2023-05-31 17:02:27.963984: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [40]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:28.185888: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 0.7019 - acc: 0.5000

2023-05-31 17:02:28.486712: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [10]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:28.534288: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 724ms/step - loss: 0.7019 - acc: 0.5000 - val_loss: 0.7723 - val_acc: 0.4000
Epoch 2/30
1/1 [==============================] - 0s 60ms/step - loss: 0.6349 - acc: 0.7000 - val_loss: 0.7679 - val_acc: 0.5000
Epoch 3/30
1/1 [==============================] - 0s 60ms/step - loss: 0.6016 - acc: 0.8000 - val_loss: 0.7261 - val_acc: 0.5000
Epoch 4/30
1/1 [==============================] - 0s 50ms/step - loss: 0.5673 - acc: 0.8000 - val_loss: 0.6974 - val_acc: 0.5000
Epoch 5/30
5/5 [==============================] - 0s 22ms/step - loss: 0.5727 - acc: 0.7200
Test Loss: 0.5726751685142517
Test Accuracy: 0.7200000286102295


In [16]:
CNN(X1_train.shape[1:], X1_train, y1_train, X1_test, y1_test, 'binary_crossentropy', 'sigmoid')

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_4 (Reshape)         (None, 251, 1)            0         
                                                                 
 conv1d_1 (Conv1D)           (None, 249, 32)           128       
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 124, 32)           0         
 g1D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 3968)              0         
                                                                 
 dense_17 (Dense)            (None, 64)                254016    
                                                                 
 dense_18 (Dense)            (None, 1)                 65        
                                                      

2023-05-31 17:02:29.233671: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [28]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:29.452658: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 0.5633 - acc: 1.0000

2023-05-31 17:02:29.736681: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [8]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:29.783633: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 706ms/step - loss: 0.5633 - acc: 1.0000 - val_loss: 0.1717 - val_acc: 1.0000
Epoch 2/30
1/1 [==============================] - 0s 65ms/step - loss: 0.1708 - acc: 1.0000 - val_loss: 0.0565 - val_acc: 1.0000
Epoch 3/30
1/1 [==============================] - 0s 52ms/step - loss: 0.0569 - acc: 1.0000 - val_loss: 0.0203 - val_acc: 1.0000
Epoch 4/30
1/6 [====>.........................] - ETA: 0s - loss: 0.0081 - acc: 1.0000

2023-05-31 17:02:30.205334: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


6/6 [==============================] - 0s 20ms/step - loss: 0.0078 - acc: 1.0000
Test Loss: 0.007847938686609268
Test Accuracy: 1.0


In [17]:
CNN(X2_train.shape[1:], X2_train, y2_train, X2_test, y2_test, 'binary_crossentropy', 'sigmoid')

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_5 (Reshape)         (None, 136, 1)            0         
                                                                 
 conv1d_2 (Conv1D)           (None, 134, 32)           128       
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 67, 32)            0         
 g1D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 2144)              0         
                                                                 
 dense_19 (Dense)            (None, 64)                137280    
                                                                 
 dense_20 (Dense)            (None, 1)                 65        
                                                      

2023-05-31 17:02:30.519924: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [18]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:30.739642: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 0.7311 - acc: 0.4444

2023-05-31 17:02:31.046309: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [5]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:31.093502: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 730ms/step - loss: 0.7311 - acc: 0.4444 - val_loss: 0.6091 - val_acc: 0.8000
Epoch 2/30
1/1 [==============================] - 0s 66ms/step - loss: 0.6393 - acc: 0.7222 - val_loss: 0.5461 - val_acc: 0.8000
Epoch 3/30
1/1 [==============================] - 0s 67ms/step - loss: 0.5861 - acc: 0.7222 - val_loss: 0.5109 - val_acc: 0.8000
Epoch 4/30
27/27 [==============================] - 0s 9ms/step - loss: 0.6044 - acc: 0.6318
Test Loss: 0.6044156551361084
Test Accuracy: 0.6318234205245972


In [18]:
def CNN_LSTM(ip_shape, X, y, x_test, y_test, loss_fn, act_fn):
    model = keras.models.Sequential([
        layers.Reshape((ip_shape[0], 1), input_shape=ip_shape),
        layers.Conv1D(32, 3, activation='relu'),
        layers.MaxPooling1D(2),
        layers.LSTM(64),
        layers.Dense(1, activation=act_fn)
    ])

    model.compile(loss=loss_fn, optimizer='adam', metrics=['acc'])
    model.summary()

    perm = np.random.permutation(len(X))
    X_sh, y_sh = X[perm], y[perm]

    train_size = int(0.8 * len(X_sh))
    val_size = len(X_sh) - train_size

    train_dataset = tf.data.Dataset.from_tensor_slices((X_sh[:train_size], y_sh[:train_size]))
    train_dataset = train_dataset.batch(64).prefetch(tf.data.AUTOTUNE)

    val_dataset = tf.data.Dataset.from_tensor_slices((X_sh[train_size:], y_sh[train_size:]))
    val_dataset = val_dataset.batch(64).prefetch(tf.data.AUTOTUNE)

    callback1 = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=3)
    callback2 = tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5', save_best_only=True)

    history = model.fit(train_dataset, shuffle=True, validation_data=val_dataset, epochs=30, callbacks=[callback1, callback2])

    test_loss, test_accuracy = model.evaluate(x_test, y_test)
    print("Test Loss:", test_loss)
    print("Test Accuracy:", test_accuracy)

In [19]:
CNN_LSTM(X_train.shape[1:], X_train, y_train, X_test, y_test, 'binary_crossentropy', 'sigmoid')


Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_6 (Reshape)         (None, 150, 1)            0         
                                                                 
 conv1d_3 (Conv1D)           (None, 148, 32)           128       
                                                                 
 max_pooling1d_3 (MaxPoolin  (None, 74, 32)            0         
 g1D)                                                            
                                                                 
 lstm_3 (LSTM)               (None, 64)                24832     
                                                                 
 dense_21 (Dense)            (None, 1)                 65        
                                                                 
Total params: 25025 (97.75 KB)
Trainable params: 25025 (97.75 KB)
Non-trainable params: 0 (0.00 Byte)
__________________

2023-05-31 17:02:32.128767: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [40]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:32.861519: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:02:33.109522: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 0.7039 - acc: 0.4000

2023-05-31 17:02:33.317453: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:02:33.505733: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [10]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:33.662035: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:02:33.775889: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 2s 2s/step - loss: 0.7039 - acc: 0.4000 - val_loss: 0.6840 - val_acc: 0.8000
Epoch 2/30
1/1 [==============================] - 0s 87ms/step - loss: 0.6963 - acc: 0.4000 - val_loss: 0.7061 - val_acc: 0.1000
Epoch 3/30
1/1 [==============================] - 0s 74ms/step - loss: 0.6900 - acc: 0.5500 - val_loss: 0.7292 - val_acc: 0.2000
Epoch 4/30
5/5 [==============================] - 0s 38ms/step - loss: 0.6977 - acc: 0.4933
Test Loss: 0.6976981163024902
Test Accuracy: 0.4933333396911621


In [20]:
CNN_LSTM(X1_train.shape[1:], X1_train, y1_train, X1_test, y1_test, 'binary_crossentropy', 'sigmoid')

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_7 (Reshape)         (None, 251, 1)            0         
                                                                 
 conv1d_4 (Conv1D)           (None, 249, 32)           128       
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 124, 32)           0         
 g1D)                                                            
                                                                 
 lstm_4 (LSTM)               (None, 64)                24832     
                                                                 
 dense_22 (Dense)            (None, 1)                 65        
                                                                 
Total params: 25025 (97.75 KB)
Trainable params: 25025 (97.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________

2023-05-31 17:02:34.701617: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [28]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:35.389318: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:02:35.642568: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:02:35.872621: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 0.6804 - acc: 1.0000

2023-05-31 17:02:36.085956: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [8]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:36.241695: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 2s 2s/step - loss: 0.6804 - acc: 1.0000 - val_loss: 0.6242 - val_acc: 1.0000


2023-05-31 17:02:36.353079: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Epoch 2/30
1/1 [==============================] - 0s 94ms/step - loss: 0.6233 - acc: 1.0000 - val_loss: 0.5693 - val_acc: 1.0000
Epoch 3/30
1/1 [==============================] - 0s 91ms/step - loss: 0.5686 - acc: 1.0000 - val_loss: 0.5157 - val_acc: 1.0000
Epoch 4/30
1/1 [==============================] - 0s 109ms/step - loss: 0.5153 - acc: 1.0000 - val_loss: 0.4625 - val_acc: 1.0000


2023-05-31 17:02:37.047984: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:02:37.157621: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


6/6 [==============================] - 1s 33ms/step - loss: 0.4664 - acc: 1.0000
Test Loss: 0.46637406945228577
Test Accuracy: 1.0


In [21]:
CNN_LSTM(X2_train.shape[1:], X2_train, y2_train, X2_test, y2_test, 'binary_crossentropy', 'sigmoid')

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_8 (Reshape)         (None, 136, 1)            0         
                                                                 
 conv1d_5 (Conv1D)           (None, 134, 32)           128       
                                                                 
 max_pooling1d_5 (MaxPoolin  (None, 67, 32)            0         
 g1D)                                                            
                                                                 
 lstm_5 (LSTM)               (None, 64)                24832     
                                                                 
 dense_23 (Dense)            (None, 1)                 65        
                                                                 
Total params: 25025 (97.75 KB)
Trainable params: 25025 (97.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________

2023-05-31 17:02:38.366915: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [18]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:39.060119: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:02:39.311314: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:02:39.539036: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 0.6888 - acc: 0.3889

2023-05-31 17:02:39.748788: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [5]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:39.903870: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 2s 2s/step - loss: 0.6888 - acc: 0.3889 - val_loss: 0.6892 - val_acc: 0.6000


2023-05-31 17:02:40.015586: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Epoch 2/30
1/1 [==============================] - 0s 101ms/step - loss: 0.6805 - acc: 0.7778 - val_loss: 0.6840 - val_acc: 0.6000
Epoch 3/30
1/1 [==============================] - 0s 83ms/step - loss: 0.6724 - acc: 0.7222 - val_loss: 0.6795 - val_acc: 0.6000
Epoch 4/30
27/27 [==============================] - 0s 13ms/step - loss: 0.6722 - acc: 0.5668
Test Loss: 0.6722450852394104
Test Accuracy: 0.5667827725410461


### Task 2: Time series classification using deep learning 2

There has been several neural network models dedicated to time series classification. Besides your own models that you developed in Lab 4, now you will develop such dedicated models by referring to some papers, and test if they indeed perform better than your rough models. There are two famous papers as follows:
 - [Convolutional neural networks for time series classification (2017)](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7870510)
 - [Time Series Classification from Scratch with Deep Neural Networks: A Strong Baseline (2017)](https://arxiv.org/abs/1611.06455)

First paper's idea is already implemented in sktime, with the name `CNNClassifier`. Second paper has three models and those are easy to develop using tensorflow. Now the task is to develop two models (MLP and FCN) in the second paper and test it together with `CNNClassifier`.

Use the same four datasets, and test sktime's `CNNClassifier` and MLP and FCN models you develop. Report test scores of three models (`CNNClassifier`, MLP, and FCN) on three datasets you chose. It would be nine scores in total. For MLP and FCN, you may need to satisfy the following requirement:

- You should use at least **two** Tensorflow callbacks when you fit your model. These can be built-in ones or your personalized callback.
- You should run the model at least 10 epochs.
- You can use the same processed datasets in Task 1. For `CNNClassifier`, as you cannot use `tf.Data`, you may put the training set directly.
- For `CNNClassifier`, you can run it with the default parameters or reduce the number of epoch (default is 2000).
- Please use the predefined test dataset to report the test scores.

Note that the main purpose of this task is to check if you can develop a similar network structure with description. If the detail of the specific part (e.g., size of one layer or some custom parameters like epoch) is missing in the paper, you can set it on your own.
 

In [22]:
def CNN(X, y, xtest, ytest, lossfn = 'binary_crossentropy'):
    X=np.reshape(X, (-1,  X.shape[1], 1))
    
    model = keras.models.Sequential([
        keras.layers.Conv1D(filters=128, kernel_size=5, activation='relu', padding='same', input_shape=X.shape[1:]),
        keras.layers.MaxPooling1D(pool_size=2),
        keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu', padding='same'),
        keras.layers.MaxPooling1D(pool_size=2),
        keras.layers.Conv1D(filters=128, kernel_size=3, activation='tanh', padding='same'),
        keras.layers.MaxPooling1D(pool_size=2),
        keras.layers.Flatten(),
        keras.layers.Dense(units=128, activation='relu'),
        keras.layers.Dense(units=1, activation='sigmoid')
        ])

    model.compile(loss=lossfn,optimizer='adam',metrics=['accuracy'])
    return model
    
def MLP(X,y,xtest,ytest,lossfn = 'binary_crossentropy'):
  
    model = keras.Sequential([
        keras.layers.Dropout(0.1),
        keras.layers.Dense(units=500,activation='relu'),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(units=500,activation='relu'),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(units=500,activation='relu'),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(units=1, activation='softmax')
        ])

    model.compile(optimizer='adam', loss=lossfn, metrics=['accuracy'])
    return model

def FCN(X,y,xtest,ytest,lossfn = 'binary_crossentropy'):
    X= X.reshape((-1, X.shape[1], 1))
    xtest = xtest.reshape((-1, xtest.shape[1], 1))

    model = keras.Sequential([
        keras.layers.Conv1D(filters=128, kernel_size=8,input_shape=X.shape[1:]),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('relu'),
        keras.layers.Conv1D(filters=256, kernel_size=5),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('relu'),
        keras.layers.Conv1D(filters=128, kernel_size=3),
        keras.layers.BatchNormalization(),
        keras.layers.Activation('relu'),
        keras.layers.GlobalAveragePooling1D(),
        keras.layers.Dense(units=1, activation='softmax')
        ])

    model.compile(optimizer='adam', loss=lossfn, metrics=['accuracy'])
    return model
models = {'CNN': CNN, 'MLP': MLP,'FCN': FCN}



In [23]:
def preprocess_data(X, y):
    perm = np.random.permutation(len(X))
    X_sh, y_sh = X[perm], y[perm]
    train_size = int(0.8 * len(X_sh))
    val_size = len(X_sh) - train_size

    train_dataset = tf.data.Dataset.from_tensor_slices((X_sh[:train_size], y_sh[:train_size]))
    train_dataset = train_dataset.shuffle(len(X_sh)).batch(32).prefetch(tf.data.AUTOTUNE)

    val_dataset = tf.data.Dataset.from_tensor_slices((X_sh[train_size:], y_sh[train_size:]))
    val_dataset = val_dataset.batch(32).prefetch(tf.data.AUTOTUNE)
    
    
    return train_dataset,val_dataset


In [24]:
def train_and_evaluate_model(model, train_data, val_data, X_test, y_test):
    
    callback1 = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=3,restore_best_weights=True)
    callback2 = tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5', save_best_only=True)
    
    history = model.fit(train_data, epochs=10, validation_data=val_data, callbacks=[callback1,callback2], shuffle=True)
    
    val_loss, val_acc = model.evaluate(val_data)
    test_loss, test_acc = model.evaluate(X_test, y_test)

    print("Validation accuracy:", val_acc)
    print("Test accuracy:", test_acc)

    return test_acc

In [25]:
for model_name, model_fn in models.items():  
    for dataset_name, dataset in datasets.items():
        X_train, y_train, X_test, y_test = dataset
        print(f"Training {model_name} on {dataset_name}")
        train_data, val_data = preprocess_data(X_train,y_train )
        model = model_fn(X_train, y_train, X_test, y_test)
        train_and_evaluate_model(model, train_data, val_data,X_test, y_test)  

Training CNN on GunPoint
Epoch 1/10


2023-05-31 17:02:41.235895: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [40,150]
	 [[{{node Placeholder/_0}}]]
2023-05-31 17:02:41.236129: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [40,150]
	 [[{{node Placeholder/_0}}]]
2023-05-31 17:02:41.525875: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - ETA: 0s - loss: 0.7126 - accuracy: 0.6250

2023-05-31 17:02:42.271469: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [10]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:02:42.325929: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 1s 589ms/step - loss: 0.7126 - accuracy: 0.6250 - val_loss: 0.6965 - val_accuracy: 0.5000
Epoch 2/10
2/2 [==============================] - 0s 109ms/step - loss: 0.6590 - accuracy: 0.7250 - val_loss: 0.6565 - val_accuracy: 0.7000
Epoch 3/10
2/2 [==============================] - 0s 41ms/step - loss: 0.6532 - accuracy: 0.5000 - val_loss: 0.6815 - val_accuracy: 0.5000
Epoch 4/10
2/2 [==============================] - 0s 59ms/step - loss: 0.5982 - accuracy: 0.8000 - val_loss: 0.6827 - val_accuracy: 0.5000
Epoch 5/10
5/5 [==============================] - 0s 41ms/step - loss: 0.6572 - accuracy: 0.5200
Validation accuracy: 0.699999988079071
Test accuracy: 0.5200000405311584
Training CNN on AroowHead
Epoch 1/10


...

6/6 [==============================] - 1s 51ms/step - loss: 0.7076 - accuracy: 1.0000
Validation accuracy: 1.0
Test accuracy: 1.0
Training FCN on ECGFiveDays
Epoch 1/10


2023-05-31 17:03:00.894963: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [18,136]
	 [[{{node Placeholder/_0}}]]
2023-05-31 17:03:00.895206: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [18,136]
	 [[{{node Placeholder/_0}}]]
2023-05-31 17:03:01.349883: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 0.6736 - accuracy: 0.2778

2023-05-31 17:03:02.663014: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [5]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:03:02.726791: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 2s 2s/step - loss: 0.6736 - accuracy: 0.2778 - val_loss: 0.7007 - val_accuracy: 0.8000
Epoch 2/10
1/1 [==============================] - 0s 158ms/step - loss: 0.5154 - accuracy: 0.2778 - val_loss: 0.7006 - val_accuracy: 0.8000
Epoch 3/10
1/1 [==============================] - 0s 142ms/step - loss: 0.4635 - accuracy: 0.2778 - val_loss: 0.7028 - val_accuracy: 0.8000
Epoch 4/10
27/27 [==============================] - 1s 18ms/step - loss: 0.6920 - accuracy: 0.5029
Validation accuracy: 0.800000011920929
Test accuracy: 0.5029035806655884


In [26]:
from sklearn.metrics import accuracy_score
clf=CNNClassifier(n_epochs=50,metrics='accuracy')
for dataset_name, dataset in datasets.items():
    X_train, y_train, X_test, y_test = dataset
    clf.fit(X_train,y_train)
    accuracy = accuracy_score(y_test, clf.predict(X_test))
    print(f"Dataset: {dataset_name}, accuracy: {accuracy}")

/Users/lekhmac/miniforge3/envs/tensenv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
2023-05-31 17:03:06.031047: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 0s 7ms/step


2023-05-31 17:03:08.587024: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
/Users/lekhmac/miniforge3/envs/tensenv/lib/python3.10/site-packages/sktime/classification/base.py:758: UserWarning: only single class label seen in y passed to fit of classifier CNNClassifier
  warn(
/Users/lekhmac/miniforge3/envs/tensenv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Dataset: GunPoint, accuracy: 0.7133333333333334
Dataset: AroowHead, accuracy: 1.0


2023-05-31 17:03:09.122230: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


42/54 [======================>.......] - ETA: 0s

2023-05-31 17:03:10.841110: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


54/54 [==============================] - 0s 3ms/step
Dataset: ECGFiveDays, accuracy: 0.8304297328687572


### Task 3: Time series classification using deep learning 3

Next, you can try to further improve your model by selecting one of the following ideas (or you can suggest your own ideas that are not trivial):
- Use Bi-Direction LSTM and CNN networks separately, create two to three layers individually, and concatenate them. This means that until the third (or second) layer, you have two different networks handling the same dataset, and after that, you concatenate the output and finish with any FCN layer. Check [this post](https://stackoverflow.com/questions/59168306/how-to-combine-lstm-and-cnn-in-timeseries-classification) to get inspired.
- Apply any sktime's transformer (not attention transformer) first to the dataset and run any deep learning model you already developed in Tasks 2 and 3. In this case, you need to choose at least two transformers and apply them together.
- Train the model on multiple similar datasets and test it on one specific test set. Check if the model can be improved if it is trained on multiple datasets (at least five datasets). However, for this, you also need to choose the similar datasets based on their classification and motivate your choise in the report (UCR repository has a specific dataset type such as **AUDIO** or **MOTION**).

Choose one model you want from the models you have developed in Tasks 1 and 2. Select one idea, try implementing it, and check if you can improve the performance. Note that you do not need to prove that the accuracy scores increase but must explain your trials. Report test scores on three datasets you chose.

In [27]:
from tensorflow.keras.layers import Input, LSTM, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Concatenate, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

#Dataset1

# Define the input shape
input_shape = (X_train.shape[1], 1)

# Define the input layer
input_layer = Input(shape=input_shape)



# Modify the LSTM layer
lstm_out = LSTM(64, activation='tanh', return_sequences=True)(input_layer)
lstm_out = GlobalMaxPooling1D()(lstm_out)

# Modify the CNN layer
cnn_out = Conv1D(64, 3, activation='relu')(input_layer)
cnn_out = MaxPooling1D(2)(cnn_out)
cnn_out = Conv1D(128, 3, activation='relu')(cnn_out)
cnn_out = MaxPooling1D(2)(cnn_out)
cnn_out = GlobalMaxPooling1D()(cnn_out)

# Concatenate the LSTM and CNN outputs
concatenated = Concatenate()([lstm_out, cnn_out])
num_classes_gunpoint = len(np.unique(y_train))


fcn_out = Dense(64, activation='relu')(concatenated)
output_layer = Dense(num_classes_gunpoint, activation='softmax')(fcn_out)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, batch_size=64, epochs=30, validation_split=0.2,
                    callbacks=[EarlyStopping(patience=3), ModelCheckpoint('best_model.h5', save_best_only=True)])

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)



Epoch 1/30


2023-05-31 17:03:12.832805: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:03:13.089580: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:03:13.475371: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 0.7106 - accuracy: 0.5556

2023-05-31 17:03:13.941089: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:03:14.061148: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 3s 3s/step - loss: 0.7106 - accuracy: 0.5556 - val_loss: 0.6483 - val_accuracy: 1.0000
Epoch 2/30
1/1 [==============================] - ETA: 0s - loss: 0.6621 - accuracy: 0.6667

/Users/lekhmac/miniforge3/envs/tensenv/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 196ms/step - loss: 0.6621 - accuracy: 0.6667 - val_loss: 0.6471 - val_accuracy: 0.6000
Epoch 3/30
1/1 [==============================] - 0s 163ms/step - loss: 0.6266 - accuracy: 0.8333 - val_loss: 0.5798 - val_accuracy: 1.0000
Epoch 4/30
1/1 [==============================] - 0s 167ms/step - loss: 0.5818 - accuracy: 0.8333 - val_loss: 0.5100 - val_accuracy: 0.8000
Epoch 5/30
1/1 [==============================] - 0s 122ms/step - loss: 0.5544 - accuracy: 0.7222 - val_loss: 0.4655 - val_accuracy: 0.8000
Epoch 6/30
1/1 [==============================] - 0s 121ms/step - loss: 0.5356 - accuracy: 0.7222 - val_loss: 0.4525 - val_accuracy: 1.0000
Epoch 7/30
1/1 [==============================] - 0s 94ms/step - loss: 0.5121 - accuracy: 0.7222 - val_loss: 0.4653 - val_accuracy: 1.0000
Epoch 8/30
1/1 [==============================] - 0s 104ms/step - loss: 0.4883 - accuracy: 0.8333 - val_loss: 0.4955 - val_accuracy: 0.8000
Epoch 9/30
27/27 [==============

In [28]:
#Dataset2

# Define the input shape
input_shape = (X1_train.shape[1], 1)

# Define the input layer
input_layer = Input(shape=input_shape)



# Modify the LSTM layer
lstm_out = LSTM(64, activation='tanh', return_sequences=True)(input_layer)
lstm_out = GlobalMaxPooling1D()(lstm_out)

# Modify the CNN layer
cnn_out = Conv1D(64, 3, activation='relu')(input_layer)
cnn_out = MaxPooling1D(2)(cnn_out)
cnn_out = Conv1D(128, 3, activation='relu')(cnn_out)
cnn_out = MaxPooling1D(2)(cnn_out)
cnn_out = GlobalMaxPooling1D()(cnn_out)

# Concatenate the LSTM and CNN outputs
concatenated = Concatenate()([lstm_out, cnn_out])
num_classes_arrowhead = len(np.unique(y1_train))


fcn_out = Dense(64, activation='relu')(concatenated)
output_layer = Dense(num_classes_arrowhead, activation='softmax')(fcn_out)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X1_train, y1_train, batch_size=64, epochs=30, validation_split=0.2,
                    callbacks=[EarlyStopping(patience=3), ModelCheckpoint('best_model.h5', save_best_only=True)])

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X1_test, y1_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)



Epoch 1/30


2023-05-31 17:03:17.066344: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:03:17.319703: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:03:17.697465: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 1.0000

2023-05-31 17:03:18.163923: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:03:18.281653: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 2s 2s/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/30
1/1 [==============================] - 0s 175ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/30
1/1 [==============================] - 0s 148ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/30
6/6 [==============================] - 0s 47ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Test Loss: 0.0
Test Accuracy: 1.0


In [29]:
#Dataset3

# Define the input shape
input_shape = (X2_train.shape[1], 1)

# Define the input layer
input_layer = Input(shape=input_shape)



# Modify the LSTM layer
lstm_out = LSTM(64, activation='tanh', return_sequences=True)(input_layer)
lstm_out = GlobalMaxPooling1D()(lstm_out)

# Modify the CNN layer
cnn_out = Conv1D(64, 3, activation='relu')(input_layer)
cnn_out = MaxPooling1D(2)(cnn_out)
cnn_out = Conv1D(128, 3, activation='relu')(cnn_out)
cnn_out = MaxPooling1D(2)(cnn_out)
cnn_out = GlobalMaxPooling1D()(cnn_out)

# Concatenate the LSTM and CNN outputs
concatenated = Concatenate()([lstm_out, cnn_out])
num_classes_ecgfivedays = len(np.unique(y2_train))


fcn_out = Dense(64, activation='relu')(concatenated)
output_layer = Dense(num_classes_ecgfivedays, activation='softmax')(fcn_out)

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X2_train, y2_train, batch_size=64, epochs=30, validation_split=0.2,
                    callbacks=[EarlyStopping(patience=3), ModelCheckpoint('best_model.h5', save_best_only=True)])

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X2_test, y2_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)



Epoch 1/30


2023-05-31 17:03:20.403143: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:03:20.669969: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:03:20.905044: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - ETA: 0s - loss: 0.6922 - accuracy: 0.5556

2023-05-31 17:03:21.347425: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:03:21.460464: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 2s 2s/step - loss: 0.6922 - accuracy: 0.5556 - val_loss: 0.8437 - val_accuracy: 0.2000
Epoch 2/30
1/1 [==============================] - 0s 167ms/step - loss: 0.6834 - accuracy: 0.4444 - val_loss: 0.6962 - val_accuracy: 0.4000
Epoch 3/30
1/1 [==============================] - 0s 173ms/step - loss: 0.6118 - accuracy: 0.7778 - val_loss: 0.5450 - val_accuracy: 1.0000
Epoch 4/30
1/1 [==============================] - 0s 186ms/step - loss: 0.5814 - accuracy: 0.6667 - val_loss: 0.4842 - val_accuracy: 0.8000
Epoch 5/30
1/1 [==============================] - 0s 168ms/step - loss: 0.5748 - accuracy: 0.6111 - val_loss: 0.4774 - val_accuracy: 1.0000
Epoch 6/30
1/1 [==============================] - 0s 134ms/step - loss: 0.5472 - accuracy: 0.6667 - val_loss: 0.5062 - val_accuracy: 1.0000
Epoch 7/30
1/1 [==============================] - 0s 104ms/step - loss: 0.5136 - accuracy: 0.8333 - val_loss: 0.5583 - val_accuracy: 0.6000
Epoch 8/30
27/27 [================

### Task 4: Time series classification using sktime

As we saw above, we can use RandomizedSearch to find optimal parameter options. However, it does not support scikit-learn's classifiers such as DecisionTree or RandomForest. However, as we tried, it is also possible to use the output of the sktime transformer (e.g., catch22) to train scikit-learn models such as RandomForest. Sktime supports this with `SklearnClassifierPipeline` to put a  scikit-learn classifier and sktime's transformer together and you need to implement it.

Pick one classifier from scikit-learn (that can be anything! e.g., Decision Tree or Logistic Regressor) and two transformers from sktime and create `SklearnClassifierPipeline.` As we tried in this lab, that can be **Rocket with RandomForest** or **Catch22 with DecisionTree**. Pick one parameter from each module (in total, three, one from the classifier and two from two transformers) and run a randomized search on the pipeline you define and report the test score of the best model found by the randomized Search. Compare your best score to the score from the same model with the default setting.

Task 4 involves a time-consuming process, so you can only choose **one dataset** to perform the task above. Also, note that you do not need to perform better by conducting a randomized search for this task (but still good to try!).

In [30]:
from sktime.transformations.series.exponent import ExponentTransformer
from sklearn.ensemble import RandomForestClassifier
from sktime.classification.compose import SklearnClassifierPipeline

pipeline =  SklearnClassifierPipeline(RandomForestClassifier(), transformers=[Rocket(), ExponentTransformer()])

In [31]:
param_distributions = {
    'estimator__classifier__max_depth': [3,6,10,13,17],
    'estimator__Rocket__normalise': [True,False],
    'estimator__ExponentTransformer__power': [1,2,3,4,5]
}

In [32]:
random_search = RandomizedSearchCV(pipeline, param_distributions, cv=3, n_iter=3, scoring='accuracy', n_jobs=-1,error_score='raise')

rocket=Rocket()
X2_train_df = X2_train
X2_test_df = X2_test

random_search.fit(X2_train_df, y2_train)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


RandomizedSearchCV(cv=3, error_score='raise',
                   estimator=SklearnClassifierPipeline(classifier=RandomForestClassifier(),
                          transformers=[Rocket(), ExponentTransformer()]),
                   n_iter=3, n_jobs=-1,
                   param_distributions={'estimator__ExponentTransformer__power': [1,
                                                                                  2,
                                                                                  3,
                                                                                  4,
                                                                                  5],
                                        'estimator__Rocket__normalise': [True,
                                                                         False],
                                        'estimator__classifier__max_depth': [3,
                                                                             6,
                                                                             10,
                                                                             13,
                                                                             17]},
                   scoring='accuracy')

In [33]:
random_search.best_params_, random_search.best_score_

({'estimator__classifier__max_depth': 13,
  'estimator__Rocket__normalise': True,
  'estimator__ExponentTransformer__power': 5},
 0.9583333333333334)

In [34]:
pipeline_best =  SklearnClassifierPipeline(RandomForestClassifier(max_depth=7), transformers=[Rocket(normalise=True), ExponentTransformer(power=2)])

In [35]:
pipeline_best.fit(X2_train_df,y2_train)
pipeline_best.score(X2_test_df,y2_test)

0.9512195121951219

In [36]:
from sktime.datatypes._panel._convert import from_2d_array_to_nested
X2_train_df = X2_train
X2_test_df = X2_test
pipeline.fit(X2_train_df,y2_train)
pipeline.score(X2_test_df,y2_test)

0.9407665505226481

### Task 5: Multivariate time series classification

Time series can be **multivariate**, which means there can be many values (= data points) describing one time point. In this task, you will use one **multivariate** dataset (**Eplipsy**) and try to run one deep learning model and one sktime model to see if those models work well on multivariate time series.

- Use Eplipsy dataset in the UCR/UEA repository.
- Run **TapNet**, which is also deep-learning based multivariate classifier from sktime, and one more classifier of your choice from sktime (which should be none deep-learning based), together with **the best tensorflow deep learning model from the previous tasks** on Eplipsy. You need to adjust the deep learning model's input layer to handle this multivariate dataset.
- Use sktime's `load_UCR_UEA_dataset` function to perform. You should use each dataset's original train/test splits.
- For the deep learning model, you should transform it using TensorFlow data API (`tf.data`) to manage your dataset and use `shuffle`, `batch`, and `prefetch` functions. This means that you need to create the validation set first. 
- For training, you need to run at least 10 epochs for your deep learning model. For TapNet, you can keep the default parameter options. 
- Report the test scores of three models on the predefined test set.
- **Do the same task on one more chosen multivariate time series dataset.**

In [37]:
X_train, y_train = load_UCR_UEA_dataset(name='Epilepsy', split="train",return_type='numpy3D')
X_test, y_test = load_UCR_UEA_dataset(name= 'Epilepsy', split="test",return_type='numpy3D')

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [38]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

train_dataset = train_dataset.shuffle(len(X_train)).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

In [39]:
from sktime.classification.deep_learning.tapnet import TapNetClassifier
tapnet= TapNetClassifier()

In [40]:
n_timesteps, n_features = X_train.shape[1], X_train.shape[2]
input_shape = (n_timesteps, n_features)
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=input_shape),
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', strides=1, padding='same'),
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu', strides=1, padding='same'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [41]:
batch_size = 32
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.shuffle(buffer_size=len(X_train))
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_ds = test_ds.batch(batch_size)
test_ds = test_ds.prefetch(tf.data.experimental.AUTOTUNE)

In [42]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(train_ds, epochs=10, validation_data=test_ds)

Epoch 1/10


2023-05-31 17:07:35.888630: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [137]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:07:35.888885: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [137]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:07:36.251148: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - ETA: 0s - loss: 1.5111 - accuracy: 0.3066

2023-05-31 17:07:37.054505: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [138]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:07:37.102675: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 2s 163ms/step - loss: 1.5111 - accuracy: 0.3066 - val_loss: 1.2754 - val_accuracy: 0.5435
Epoch 2/10
5/5 [==============================] - 0s 43ms/step - loss: 1.2159 - accuracy: 0.4964 - val_loss: 1.1590 - val_accuracy: 0.5362
Epoch 3/10
5/5 [==============================] - 0s 41ms/step - loss: 0.9701 - accuracy: 0.6058 - val_loss: 1.0624 - val_accuracy: 0.6087
Epoch 4/10
5/5 [==============================] - 0s 21ms/step - loss: 0.7754 - accuracy: 0.7445 - val_loss: 0.9867 - val_accuracy: 0.6449
Epoch 5/10
5/5 [==============================] - 0s 25ms/step - loss: 0.6500 - accuracy: 0.7956 - val_loss: 0.9152 - val_accuracy: 0.6159
Epoch 6/10
5/5 [==============================] - 0s 22ms/step - loss: 0.5907 - accuracy: 0.8175 - val_loss: 0.8453 - val_accuracy: 0.6667
Epoch 7/10
5/5 [==============================] - 0s 24ms/step - loss: 0.4144 - accuracy: 0.8759 - val_loss: 0.8219 - val_accuracy: 0.6594
Epoch 8/10
5/5 [=====================

In [43]:
tapnet.fit(X_train, y_train)

/Users/lekhmac/miniforge3/envs/tensenv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
2023-05-31 17:07:42.396260: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:07:43.379360: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:07:45.943277: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


TapNetClassifier()

In [44]:
tapnet_score = tapnet.score(X_test, y_test)
loss, acc = model.evaluate(test_dataset)

2023-05-31 17:53:07.444425: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-05-31 17:53:07.790683: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/5 [=====>........................] - ETA: 0s - loss: 0.9910 - accuracy: 0.5312

2023-05-31 17:53:09.332459: I tensorflow/core/common_runtime/executor.cc:1210] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [138]
	 [[{{node Placeholder/_1}}]]
2023-05-31 17:53:09.341794: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


5/5 [==============================] - 0s 36ms/step - loss: 0.7214 - accuracy: 0.6667


In [45]:
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=10,
    cv=5,
    scoring="accuracy",
    random_state=42,
    n_jobs=-1,
)

random_search.fit(X_train, y_train)
best_score = random_search.best_score_

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [46]:
print(f"Test score of the best  model: {best_score:.4f}")
print(f"Test score of the tapnet: {tapnet_score:.4f}")
print(f"Test score of the deep learning model: {acc:.4f}")

Test score of the best  model: 0.9635
Test score of the tapnet: 0.9710
Test score of the deep learning model: 0.6667
